In [1]:
from premise import *
from datapackage import Package
import bw2io as bw
import bw2data
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

Loading variables from /Users/romain/Github/premise/dev/variables.yaml
USER_DATA_BASE_DIR: test


In [2]:
ndb = NewDatabase(
    scenarios=[
        {"model":"remind", "pathway":"SSP2-NPi", "year":2050},
    ],
    source_db="ecoinvent-3.10-cutoff", # <-- name of the database in the BW2 project. Must be a string.
    source_version="3.10", # <-- version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
    key="tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=",
    biosphere_name="ecoinvent-3.10-biosphere",
    keep_imports_uncertainty=True
)

premise v.(2, 3, 0, 'dev0')
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your bw2 project:                                         |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-----

In [3]:
ndb.update("electricity")

Processing scenarios for sector 'electricity':   0%| | 0/1 [00:00<?, ?

Processing scenarios for sector 'electricity': 100%|█| 1/1 [00:15<00:0

Done!



In [38]:
db, df, cols = ndb.write_decomposition_db_to_brightway()

Dropped 144 duplicate(s).


In [39]:
import numpy as np
# add as many columns to df as there are columns in cols
for col in cols:
    df[col] = np.nan

In [40]:
import numpy as np
import pandas as pd

# Ensure correct indexing for fast lookups
df["from reference product"] = df["from reference product"].fillna("None")
df["from location"] = df["from location"].fillna("None")
df["from categories"] = df["from categories"].fillna("None")

index_cols = [
    "to activity name",
    "to reference product",
    "to location",
    "from activity name",
    "from reference product",
    "from location",
    "from categories",
    "flow type",
]

df.set_index(index_cols, inplace=True)
df.sort_index(inplace=True)  # **Important: Sort index to avoid lexsort warnings**

In [41]:
# Process all exchanges in bulk
updates = []
col = df.columns[-(len(cols) + 1)]
for act in ndb.scenarios[0]["database"]:
    for exc in act["exchanges"]:
        if any(exc.get(k, 0) != 0 for k in cols):
            lookup_key = (
                act["name"],
                act["reference product"],
                act["location"],
                exc["name"],
                exc.get("product", "None"),
                exc.get("location", "None"),
                exc.get("categories", "None"),
                exc["type"],
            )

            if lookup_key in df.index:

                original_values = df.loc[lookup_key, "original"]

                # If it's a Series, take the first value
                if isinstance(original_values, pd.Series):
                    original_values = original_values.iloc[0]

                if original_values == 0:
                    original_values = df.loc[lookup_key, col]
                    if isinstance(original_values, pd.Series):
                        original_values = original_values.iloc[0]

                new_values = {col: exc[col] * original_values for col in cols if col in exc}
                updates.append((lookup_key, new_values))

# Convert updates into a DataFrame for fast merging
update_df = pd.DataFrame.from_dict(
    {key: values for key, values in updates}, orient="index", columns=cols
)

# Ensure index alignment
update_df.index.names = df.index.names

# Apply updates using `update()` for efficiency
df.update(update_df)

cols = ["original"] + cols

# Cascade values down columns
for i in range(len(cols) - 1):
    current_col = cols[i]
    next_col = cols[i + 1]

    # Fill next column with current column's values where missing
    df[next_col] = df[next_col].fillna(df[current_col])

# Reset index if needed
df.reset_index(inplace=True)

In [42]:
df.to_excel("df.xlsx")

In [5]:
for act in ndb.database:
    for exc in act["exchanges"]:
        if "electricity - efficiency" in exc:
            print("yes")

In [6]:
ndb.database[0]["exchanges"][0].keys()

dict_keys(['name', 'product', 'unit', 'location', 'type', 'amount'])

In [10]:
ndb.scenarios[0]["database"][0]["exchanges"][2].keys()

dict_keys(['name', 'unit', 'categories', 'type', 'amount', 'input'])

In [5]:
from premise.utils import load_database
ndb.scenarios[0] = load_database(ndb.scenarios[0])

In [6]:
keys = []
for act in ndb.scenarios[0]["database"]:
    for e in act["exchanges"]:
        for k in e.keys():
            if k not in keys:
                keys.append(k)
list(set(keys))

['electricity - efficiency',
 'input',
 'Comment',
 'scale',
 'minimum',
 'simapro category',
 'production volume',
 'comment',
 'reference product',
 'ub',
 'formula',
 'categories',
 'electricity - market',
 'u2',
 'loc',
 'product',
 'allocation',
 'normalization',
 'u4',
 'system model',
 'database',
 'unit',
 'maximum',
 'name',
 'tag',
 'type',
 'original_amount',
 'simapro name',
 'dataset name',
 'shape',
 'electricity - hydro - correction',
 'u1',
 'electricity - air emissions',
 'negative',
 'u6',
 'u5',
 'u3',
 'amount',
 'input type',
 'uncertainty type',
 'pedigree',
 'location']

In [8]:
[
    k 
    for a in ndb.scenarios[0]["database"] 
    for e in a["exchanges"] 
    for k in e.keys()
    if k not in (
        'input', 'Comment', 'scale', 'minimum', 'simapro category', 
        'production volume', 'comment', 'reference product', 'ub', 
        'formula', 'categories', 'u2', 'loc', 'product', 'allocation', 
        'normalization', 'u4', 'system model', 'database', 'unit', 
        'maximum', 'name', 'tag', 'type', 'original_amount', 
        'simapro name', 'dataset name', 'shape', 'u1', 'negative', 
        'u6', 'u5', 'u3', 'amount', 'input type', 'uncertainty type', 
        'pedigree', 'location'
    )
]


['electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - air emissions',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electricity - efficiency',
 'electrici